In [1]:
import cv2
from pathlib import Path
from random import *
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage.feature import hog
from imutils import face_utils
import dlib
import os
import pickle
np.random.seed(1000)


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
# loading data
frames = []
labels = []
for file in os.listdir('output/'):
    if file[-10:] == 'frames.pkl':
        with open('output/'+file, 'rb') as f:
            frames.append(pickle.load(f))
    elif file[-10:] == 'labels.pkl':
        with open('output/'+file, 'rb') as f:
            labels.append(pickle.load(f))

In [4]:
print(len(frames), len(labels))

9067 9067


In [5]:
from sklearn.model_selection import train_test_split
train_clips, test_clips, train_clips_labels, test_clips_labels = \
    train_test_split(frames, labels, test_size=0.2, random_state=42)

In [6]:
train_images, test_images, train_labels, test_labels = [], [], [], []

for clip, label in zip(train_clips, train_clips_labels):
    try:
        train_images, train_labels = train_images + clip, train_labels + [label[0]] * len(clip)
    except:
        continue

for clip, label in zip(test_clips, test_clips_labels):
    try:
        test_images, test_labels = test_images + clip, test_labels + [label[0]] * len(clip)
    except:
        continue
    
print(len(train_images), len(train_labels), len(test_images), len(test_labels))

1944475 1944475 482212 482212


In [8]:
# applying adaptive histogram equalization to enhance images
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
for i in range(len(train_images)):
    train_images[i] = clahe.apply(train_images[i])
for i in range(len(test_images)):
    test_images[i] = clahe.apply(test_images[i])

In [9]:
train_images, test_images, train_labels, test_labels = np.asarray(train_images), np.asarray(test_images), np.asarray(train_labels), np.asarray(test_labels)

In [10]:
test_images = np.expand_dims(test_images, axis=3)
train_images = np.expand_dims(train_images, axis=3)

In [32]:
model = keras.Sequential([
    keras.layers.Conv2D(filters=288, input_shape=(48,48,1), kernel_size=3, padding='valid', activation='elu'),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=288, kernel_size=1, padding='valid', activation='elu'),
    keras.layers.Conv2D(filters=288, kernel_size=3, strides=2, padding='valid', activation='elu'),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Dropout(0.5),
    keras.layers.Conv2D(filters=144, kernel_size=3, padding='valid', activation='elu'),
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=144, kernel_size=1, padding='valid', activation='elu'),
    keras.layers.Conv2D(filters=144, kernel_size=1, padding='valid', activation='elu'),
    keras.layers.MaxPooling2D(pool_size=3, strides=2),
    keras.layers.Dropout(0.5),
    keras.layers.Conv2D(filters=48, kernel_size=3, padding='valid', activation='elu'),   
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=48, kernel_size=3, padding='valid', activation='elu'),   
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=48, kernel_size=3, padding='valid', activation='elu'),   
    keras.layers.BatchNormalization(axis=-1),
    keras.layers.Conv2D(filters=4, kernel_size=1, padding='valid', activation='elu'),
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(4, activation='softmax')
])

In [33]:
model.compile(optimizer='adamax',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [34]:
# setting balanced weighting based on the relative number of points for each class
weights = {0:8.46498598, 1:0.53138737}
print(weights)

{0: 8.46498598, 1: 0.53138737}


In [35]:
model.fit(train_images, train_labels, epochs=2, class_weight=weights, batch_size=50)

  ...
    to  
  ['...']
Train on 1944475 samples
Epoch 1/2
1944475/1944475 [==============================] - 2018s 1ms/sample - loss: 0.3818 - accuracy: 0.8235
Epoch 2/2
1944475/1944475 [==============================] - 2068s 1ms/sample - loss: 0.2380 - accuracy: 0.8935


In [36]:
test_loss, test_acc = model.evaluate(test_images,  test_labels)

482212/482212 [==============================] - 169s 351us/sample - loss: 0.4059 - accuracy: 0.8560
